***TF Train API***

https://www.tensorflow.org/get_started/get_started

An ***optimizer*** changes tf variables to minimize a loss function. The simplest is ***gradient descent***:

In [12]:
import tensorflow as tf

# Linear model:
W = tf.Variable([.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
linear_model = W * x + b

# Loss function:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)

# Gradient descent optimizer:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# Get a TF session and initialize the variables:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

# Run the graph:
for i in range(1000):
    sess.run(train, {x: x_train, y: y_train})
    if i % 333 == 0:
        print("Iteration " + str(i) + ":" + "\t" + str(sess.run([W, b, loss], {x: x_train, y: y_train})))


Iteration 0:	[array([-0.21999997], dtype=float32), array([-0.456], dtype=float32), 4.0181446]
Iteration 333:	[array([-0.99050313], dtype=float32), array([ 0.97207808], dtype=float32), 0.00052083354]
Iteration 666:	[array([-0.9998281], dtype=float32), array([ 0.99949461], dtype=float32), 1.7062862e-07]
Iteration 999:	[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32), 5.6999738e-11]


***TF Contrib Learn API***

Same linear regression problem with tf.contrib.learn:

In [ ]:
import tensorflow as tf
import numpy as np

# tf.contrib abstraction for features (placeholders) and estimator:
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

# Train and test data:
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

# Functions to pass data into estimator:
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x_train}, y_train, batch_size=4, num_epochs=1000)
eval_input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x_eval}, y_eval, batch_size=4, num_epochs=1000)

# Update the model (estimator) with the training data:
estimator.fit(input_fn=input_fn, steps=1000)

# Evaluate the model:
train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)
print("Train loss: %r" % train_loss)
print("Eval loss: %r" % eval_loss)

The example above uses a predefined Linear Regression model. You can also make a custom model:

In [37]:
#######################################################
# Define model that takes features, labels, and mode: #
#######################################################
def model(features, labels, mode):
    
    # Linear model and loss:
    W = tf.get_variable("W", shape=[1], dtype=tf.float64)
    b = tf.get_variable("b", shape=[1], dtype=tf.float64)
    y = W * features["x"] + b
    loss = tf.reduce_sum(tf.square(y - labels))
    
    # Training subgraph:
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss), tf.assign_add(global_step, 1))
    
    # Return a "ModelFnOps" instance:
    return tf.contrib.learn.ModelFnOps(mode=mode, predictions=y, loss=loss, train_op=train)

###########################################
# As before, but using this custom model: #
###########################################

# tf.contrib abstraction for features (placeholders) and estimator:
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]
estimator = tf.contrib.learn.Estimator(model_fn=model)

# Train and test data:
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

# Functions to pass data into estimator:
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x_train}, y_train, batch_size=4, num_epochs=1000)
eval_input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x_eval}, y_eval, batch_size=4, num_epochs=1000)
    
# Update the model (estimator) with the training data:
estimator.fit(input_fn=input_fn, steps=1000)

# Evaluate the model:
train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)
print("Train loss: %r" % train_loss)
print("Eval loss: %r" % eval_loss)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_is_chief': True, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002768FD50518>, '_task_type': None, '_num_worker_replicas': 0, '_evaluation_master': '', '_environment': 'local', '_tf_random_seed': None, '_master': '', '_keep_checkpoint_max': 5, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\Isaac\\AppData\\Local\\Temp\\tmpmx6aij98', '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Isaac\AppData\Local\Temp\tmpmx6aij98\model.ckpt.
INFO:tensorflow:loss = 159.417773707, step = 1
INFO:tensorflow:global_step/sec: 621.38
INFO:tensorflow:loss = 0.056955600223, step = 101 (0.163 sec)
INFO:tensorflow:g